In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [149]:
df = pd.read_csv("Cleaned_car.csv")
X = df.drop(columns=['Price'])
y = df.Price
X.head()

,name,company,year,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,45000.0,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,40.0,Diesel
2,Hyundai Grand i10,Hyundai,2014,28000.0,Petrol
3,Ford EcoSport Titanium,Ford,2014,36000.0,Diesel
4,Ford Figo,Ford,2012,41000.0,Diesel


In [90]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=44)

In [107]:
from sklearn.base import BaseEstimator , TransformerMixin
from sklearn.preprocessing import StandardScaler , MinMaxScaler , MaxAbsScaler
class Standard_Car(BaseEstimator, TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        sc= StandardScaler()
        X['kms_driven'] = sc.fit_transform(X[['kms_driven']])
        return X

In [108]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe = OneHotEncoder()
from sklearn.pipeline import make_pipeline
ohe.fit(X[["fuel_type", "company","name"]])
col_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_) , 
               ["fuel_type", "company","name"]), remainder="passthrough"
                                   )
def get_model(model):
    model = make_pipeline(Standard_Car(),col_trans, model)
    model.fit(X_train, y_train)
    train_score= model.score(X_train, y_train)
    test_score = model.score(X_test,y_test)
    return model, train_score, test_score

# LR model

In [109]:
from sklearn.linear_model import LinearRegression
lr = get_model(LinearRegression())
lr[1:]

(0.9472407667684005, 0.7486253249899959)

# RFR

In [111]:
from sklearn.ensemble import RandomForestRegressor
rfr = get_model(RandomForestRegressor(n_estimators=200))
rfr[1:]

(0.946290596322971, 0.7387701549721004)

# KNN

In [77]:
from sklearn.neighbors import KNeighborsRegressor
knn = get_model(KNeighborsRegressor(n_neighbors=3))
knn[1:]

(0.8006701832974726, 0.5239046480648049)

# SVM

In [78]:
from sklearn.svm import SVR
svm = get_model(SVR(kernel='sigmoid'))
svm[1:]

(-0.07927946590959745, -0.05333980767215163)

In [113]:
import pickle as pkl
pkl.dump(lr[0], open('model.pkl', 'wb'))